<!-- 99.4% validation accuracy
Less than 20k Parameters
You can use anything from above you want. 
Less than 20 Epochs
Have used BN, Dropout, a Fully connected layer, have used GAP. 
To learn how to add different things we covered in this session, you can refer to this code: https://www.kaggle.com/enwei26/mnist-digits-pytorch-cnn-99 DONT COPY ARCHITECTURE, JUST LEARN HOW TO INTEGRATE THINGS LIKE DROPOUT, BATCHNORM, ETC. -->

# Step 1 
1. 99.4% validation accuracy
2. Less than 20k Parameters
3. You can use anything from above you want.
4. Less than 20 Epochs
5. Have used BN, Dropout, a Fully connected layer, have used GAP.
6. To learn how to add different things we covered in this session, you can refer to this code: https://www.kaggle.com/enwei26/mnist-digits-pytorch-cnn-99 DONT COPY ARCHITECTURE, JUST LEARN HOW TO INTEGRATE THINGS LIKE DROPOUT, BATCHNORM, ETC.

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

/home/shivam13juna/Documents/virtual_envs/capstone/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) # input is 1 x 28 x 28 and output is 8 x 28 x 28
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)# input is 8 x 28 x 28 and output is  16 x 28 x 28
        self.bn2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1) # input is 16 x 14 x 14 and output is 32 x 14 x 14
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1) # input is 32 x 14 x 14 and output is 32 x 14 x 14
        self.bn4 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.10)
        self.conv6 = nn.Conv2d(32, 10, 7) # input is 32 x 7 x 7 and output is 10 x 1 x 1
        self.bn6 = nn.BatchNorm2d(10)
        # self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) # input is 10 x 7 x 7 and output is 10 x 1 x 1
        self.fc = nn.Linear(10, 10)

    def forward(self, x):
        x = self.dropout(self.pool1(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x))))))))
        x = self.dropout(self.pool2(self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x))))))))
        x = self.bn6(self.conv6(x))
        # x = self.avgpool(x)
        x = x.view(-1,10)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)



from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 28, 28]           1,168
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
           Dropout-6           [-1, 16, 14, 14]               0
            Conv2d-7           [-1, 32, 14, 14]           4,640
       BatchNorm2d-8           [-1, 32, 14, 14]              64
            Conv2d-9           [-1, 32, 14, 14]           9,248
      BatchNorm2d-10           [-1, 32, 14, 14]              64
        MaxPool2d-11             [-1, 32, 7, 7]               0
          Dropout-12             [-1, 32, 7, 7]               0
           Conv2d-13             [-1, 10, 1, 1]          15,690
      BatchNorm2d-14             [-1, 1

In [4]:


torch.manual_seed(1)
batch_size = 16

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [7]:

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.11365287005901337 batch_id=3749: 100%|██████████| 3750/3750 [00:13<00:00, 275.30it/s] 



Test set: Average loss: 0.0481, Accuracy: 9878/10000 (98.780%)



loss=0.09214647859334946 batch_id=3749: 100%|██████████| 3750/3750 [00:14<00:00, 262.67it/s]  



Test set: Average loss: 0.0305, Accuracy: 9913/10000 (99.130%)



loss=0.015157821588218212 batch_id=3749: 100%|██████████| 3750/3750 [00:14<00:00, 259.09it/s] 


...


Test set: Average loss: 0.0165, Accuracy: 9943/10000 (99.430%)



loss=0.025230715051293373 batch_id=3749: 100%|██████████| 3750/3750 [00:22<00:00, 167.25it/s]  



Test set: Average loss: 0.0184, Accuracy: 9940/10000 (99.400%)



loss=0.03274239972233772 batch_id=3749: 100%|██████████| 3750/3750 [00:21<00:00, 172.43it/s]   



Test set: Average loss: 0.0184, Accuracy: 9934/10000 (99.340%)

